In [1]:
%pip install deap update_checker tqdm stopit
%pip install --upgrade -q wandb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb
from wandb.keras import WandbCallback

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

True

In [3]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv


In [5]:
home_directory = '/kaggle/input/playground-series-s3e17/'
train = pd.read_csv(home_directory+'train.csv')
test = pd.read_csv(home_directory+'test.csv')
sample_submission = pd.read_csv(home_directory+'sample_submission.csv')

In [6]:
# Create new features by comparing other ones
def additional_features(df):

    # Calculate 'TemperatureDifference' as the difference between 'Process temperature [K]' and 'Air temperature [K]'
    df['temperature_difference'] = df['process_temperature__k'] - df['air_temperature__k']
    
    # Calculate 'TemperatureRatio' as the ratio of 'Process temperature [K]' to 'Air temperature [K]'
    df['temperature_ratio'] = df['process_temperature__k'] / df['air_temperature__k']
    
    # Calculate 'Power' as the product of 'Torque [Nm]' and 'Rotational speed [rpm]'
    df['power'] = df['torque__nm'] * df['rotational_speed__rpm']
        
    # Calculate 'TotalFailures' as the sum of 'TWF', 'HDF', 'PWF', 'OSF', and 'RNF'
    df['tf'] = df[['twf', 'hdf', 'pwf', 'osf', 'rnf']].sum(axis=1)
    
    return df

In [7]:
from sklearn.preprocessing import OneHotEncoder

def sklearn_ohe(df, cat_column = 'type'):
    # # Create an OneHotEncoder object
    ohe = OneHotEncoder()
    
    # # Fit the OneHotEncoder object to the categorical column
    ohe.fit(df[[cat_column]])

    # Transform the categorical column
    transformed = ohe.transform(df[[cat_column]])

    # # Convert the transformed data to a dataframe
    encoded_df = pd.DataFrame(transformed.toarray(), columns=ohe.get_feature_names_out())

    # # Save the encoded dataframe back to the original dataframe
    return df.join(encoded_df)

In [8]:
X = train.copy()
# X.columns = [''.join(filter(str.isalnum, i)).lower() for i in X.columns]
X.columns = [i.replace(' ','_').replace("[",'_').replace("]",'').lower() for i in X.columns]

X = additional_features(X)

X = sklearn_ohe(X, cat_column = 'type')
X['product_id'] = [i[1:] for i in X['product_id']]
X['product_id'] = X['product_id'].astype('int')

In [9]:
# Data Prep
X.drop(['id','type'], axis=1, inplace=True)

# for i in ['type']:
#     X[i] = X[i].astype('category')

y = X.pop('machine_failure')

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
# help(TPOTClassifier)

In [11]:
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42, n_jobs=-1, max_time_mins=60)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')

Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.35000000000000003, min_samples_leaf=18, min_samples_split=9, n_estimators=100)
0.996628307556989


In [12]:
from tpot.export_utils import (
    export_pipeline,
    expr_to_tree,
    generate_pipeline_code,
    set_param_recursive,
)

# tpot = TPOTRegressor(generations=25, ...)
# tpot.fit()


for pipeline, pipeline_scores in zip(tpot._pareto_front.items, reversed(tpot._pareto_front.keys)):
    idx = tpot._pareto_front.items.index(pipeline)
    pareto_front_pipeline_score = pipeline_scores.wvalues[1]
    sklearn_pipeline_str = generate_pipeline_code(
        expr_to_tree(pipeline, tpot._pset), tpot.operators
    )
    to_write = export_pipeline(
        pipeline,
        tpot.operators,
        tpot._pset,
        tpot._imputed,
        pareto_front_pipeline_score,
        tpot.random_state,
    )
    print(to_write, '\n----------------------------------------------------------------------------\n\n')

import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=42)

# Average CV score on the training set was: 0.9960693821593101
exported_pipeline = ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.35000000000000003, min_samples_leaf=18, min_samples_split=9, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
 
------

In [13]:
my_dict = list(tpot.evaluated_individuals_.items())

model_scores = pd.DataFrame()
for model in my_dict:
    model_name = model[0]
    model_info = model[1]
    cv_score = model[1].get('internal_cv_score')  # Pull out cv_score as a column (i.e., sortable)
    model_scores = model_scores.append({'model': model_name,
                                        'cv_score': cv_score,
                                        'model_info': model_info,},
                                       ignore_index=True)

model_scores = model_scores.sort_values('cv_score', ascending=False)

/tmp/ipykernel_20/414146662.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_scores = model_scores.append({'model': model_name,
/tmp/ipykernel_20/414146662.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_scores = model_scores.append({'model': model_name,
/tmp/ipykernel_20/414146662.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_scores = model_scores.append({'model': model_name,
/tmp/ipykernel_20/414146662.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_scores = model_scores.append({'model': model_name,
/tmp/ipykernel_20/414146662.py:8: FutureWarning: The frame.append method is deprecated and will 

In [14]:
# tpot.evaluated_individuals_.items()

In [15]:
# model[1].get('internal_cv_score')

In [16]:
# model_scores

In [17]:
model_scores.loc[model_scores['cv_score']==-np.inf, 'cv_score']=0

In [18]:
# import re

# expr = re.compile("(?:^|])([^[\]]+)(?:\[|$)")

# # print(expr.findall("myexpr[skip this[and this]]another[and skip that too]"))

# for idx, row,in model_scores.iterrows():
#     print(idx, row[0], row[1], row[2])
# #     print(expr.findall(row[0]))

In [19]:
for idx, row,in model_scores.iterrows():
    # Initialize W&B run
    CONFIG = dict (
        train_val_split = 0.2,
        infra = "Kaggle",
        competition = 's3e17-tpot',
        name = row[0],
        type = 'experiments',
        background = row[2]
    )
    
    with wandb.init(project='s3e17-tpot', 
                     config=CONFIG,
                     group='tpot', 
                     job_type='train',
                    settings=wandb.Settings(start_method='fork')
                   ) as run:
        wandb.summary["cv"] = row[1]
        wandb.log({
        "cv": row[1]})

wandb: Currently logged in as: howt51. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230623_130649-0bextmg6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run worthy-cosmos-8
wandb: ⭐️ View project at https://wandb.ai/howt51/s3e17-tpot
wandb: 🚀 View run at https://wandb.ai/howt51/s3e17-tpot/runs/0bextmg6
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb: cv ▁
wandb: 
wandb: Run summary:
wandb: cv 0.99607
wandb: 
wandb: 🚀 View run worthy-cosmos-8 at: https://wandb.ai/howt51/s3e17-tpot/runs/0bextmg6
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230623_130649-0bextmg6/logs
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230623_130727-s2fs47u8
wandb: Run `wandb offline` to turn off syncing.
wand